# Orbit visualizer
Plot trajectory given coordinates of observations in csv format.

## Input
Each line contains a time of observation, the coordinates of observations in the sky ("RA" and "dec"), and three position coordinates which determine the shape of the planets orbit (Distance to Earth/Sun and "Solar Elongation").

## Imports

In [1]:
import numpy as np
import pandas as pd

#Data viz
from gtda.plotting import plot_point_cloud

## Sky Data

In [173]:
df = pd.read_csv('Mercury 2020.csv')

#first three entries: time and sky coordinates
skyData = df.to_numpy()[:,:3]

#accounting for wraparound
for i in range(len(skyData)-1):
    if skyData[i+1,1]<10 and skyData[i,1]>350:
        for j in range(i+1,len(skyData)): skyData[j,1]+=360         

plot_point_cloud(data)

## Positional Data

In [176]:
solarData = df.to_numpy()[:,[0,3,4,5]]

#We change to polar coordinates, in such a way that the angular coordinate is always increasing (as desired)
def convertToPolar(positionalData):
    polarData = [] #Each entry is [time, radius, angle]
    for i in range(len(data2)):
        d = data2[i]
        #Sin rule
        angle = np.arcsin( (d[1]/d[2]) * np.sin(d[3]*np.pi/180) )
        if np.isnan(angle): continue

        if i>0:
            lastAngle = polarData[-1][2]
            if angle < lastAngle:
                phase = (lastAngle//(np.pi/2))

                #phase shift
                if ((phase%4 == 0 or phase%4 == 3) and np.sin(angle) < np.sin(lastAngle)) or ((phase%4 == 1 or phase%4 == 2) and np.sin(angle) > np.sin(lastAngle)): 
                    phase += 1
                if phase%4 == 0: angle += phase*(np.pi/2)
                if phase%4 == 1: angle = (phase+1)*(np.pi/2) - angle
                if phase%4 == 2: angle = phase*(np.pi/2) - angle
                if phase%4 == 3: angle += (phase+1)*(np.pi/2)
        polarData.append([i, d[2], angle])
    return np.array(polarData)

polarData = convertToPolar(solarData)

plot_point_cloud(polarData[:,[0,2]]) #angle vs. time

C:\Users\openb\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning:

invalid value encountered in arcsin



In [171]:
cartesianData = np.array([[d[0],d[1]*np.cos(d[2]), d[1]*np.sin(d[2])] for d in polarData])
plot_point_cloud(cartesianData)